# Comparative Study of Terrain Generators
This notebook generates and compares terrain heightmaps produced by three different fractal algorithms: Diamond-Square (DS), Fractal Brownian Motion (FBM), and a Hybrid mix of both. We keep the mean elevation constant across maps, then analyze their elevation distributions, power spectra, and estimate fractal (box-counting) dimensions.

## 1. Imports and Helper Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from heightmap_generators import generate_heightmap
from scipy import fftpack

def equalize_mean(Z, target_mean=0.5):
    Zn = (Z - Z.min()) / (Z.max() - Z.min())
    curr_mean = Zn.mean()
    Z_eq = np.clip(Zn + (target_mean - curr_mean), 0, 1)
    return Z_eq

def radial_power_spectrum(Z):
    F = fftpack.fftshift(fftpack.fft2(Z))
    P = np.abs(F)**2
    ny, nx = Z.shape
    cy, cx = ny // 2, nx // 2
    y, x = np.indices(P.shape)
    r = np.hypot(x - cx, y - cy).astype(int)
    tbin = np.bincount(r.ravel(), P.ravel())
    nr = np.bincount(r.ravel())
    radial = tbin / np.maximum(nr, 1)
    freqs = np.arange(len(radial))
    return freqs[1:], radial[1:]

def box_count_dim(Z, sizes=None):
    bin_map = Z > Z.mean()
    n = Z.shape[0]
    if sizes is None:
        max_exp = int(np.log2(n))
        sizes = [2**i for i in range(1, max_exp)]
    counts = []
    for size in sizes:
        num = 0
        grid_count = n // size
        for i in range(grid_count):
            for j in range(grid_count):
                if bin_map[i*size:(i+1)*size, j*size:(j+1)*size].any():
                    num += 1
        counts.append(num)
    logs = np.log(counts)
    logs_inv = np.log(1/np.array(sizes))
    slope, _ = np.polyfit(logs_inv, logs, 1)
    return slope


## 2. Generate and Equalize Terrains

In [ ]:
# Parameters
size = 129
seed = 42

# Generate heightmaps
Z_ds = generate_heightmap('diamond-square', size=size, seed=seed, roughness=1.0)
Z_fbm = generate_heightmap('fbm', size=size, seed=seed, octaves=6, persistence=0.5, lacunarity=2.0, scale=50.0)
# Hybrid: average of DS and FBM
Z_hyb = 0.5 * Z_ds + 0.5 * Z_fbm

# Equalize mean elevation
Z_ds_eq = equalize_mean(Z_ds)
Z_fbm_eq = equalize_mean(Z_fbm)
Z_hyb_eq = equalize_mean(Z_hyb)

## 3. Elevation Histograms

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(15,4))
for ax, Z, title in zip(axs, [Z_ds_eq, Z_fbm_eq, Z_hyb_eq], ['Diamond-Square','FBM','Hybrid']):
    ax.hist(Z.ravel(), bins=50)
    ax.set_title(f"{title} Elevation Histogram")
    ax.set_xlabel('Elevation')
    ax.set_ylabel('Frequency')
plt.tight_layout()
plt.show()

## 4. Power Spectrum (Log-Log)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(15,4))
for ax, Z, title in zip(axs, [Z_ds_eq, Z_fbm_eq, Z_hyb_eq], ['Diamond-Square','FBM','Hybrid']):
    f, P = radial_power_spectrum(Z)
    ax.loglog(f, P)
    ax.set_title(f"{title} Power Spectrum")
    ax.set_xlabel('Frequency')
    ax.set_ylabel('Power')
plt.tight_layout()
plt.show()

## 5. Box-Counting Dimension Estimates

In [ ]:
dims = { 'Diamond-Square': box_count_dim(Z_ds_eq),
         'FBM': box_count_dim(Z_fbm_eq),
         'Hybrid': box_count_dim(Z_hyb_eq) }
import pandas as pd
df = pd.DataFrame.from_dict(dims, orient='index', columns=['Box-Counting Dimension'])
df

## 6. Discussion
Based on the elevation distributions, power spectra slopes, and fractal dimensions above, discuss which algorithm best matches real-world terrain statistics. For example, compare spectral slope values to those observed in natural DEMs (β≈2.5–3.0) and fractal dimensions (≈2.2–2.7).